<a href="https://colab.research.google.com/github/seungyeon38/Webtoon_recommendation_system/blob/master/%EC%BD%94%EC%8B%B8%EC%9D%B8%EC%9C%A0%EC%82%AC%EB%8F%84%EC%A0%81%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
import numpy as np
import pandas as pd
import string
string.punctuation
from math import log
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag
import re
from tqdm import tqdm
from gensim.summarization import keywords
from textblob import TextBlob

!pip install gensim


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [75]:
comic = pd.read_csv('./sample_data/Webtoon Dataset.csv')

In [76]:
input_comic = input()

Let's Play


In [77]:
# selected_comic = comic.loc[(comic.Name == input_comic)]

# print(selected_comic['Name'])

In [78]:
# 축약형 
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", "she's": "she is"}

def regularize_contractions(text):

    contractionfree_list = []
    
    for word in text.split(" "):
      if word in contractions.keys():
        contractionfree_list.append(contractions[word])
      else :
        contractionfree_list.append(word)
      
    return " ".join(contractionfree_list)


In [79]:
#defining the function to remove punctuation
# 글자가 puctuation에 해당하면 " "를, 해당하지 않으면 글자 그대로

def remove_punctuation(text):
    punctuationfree_list = []

    for word in text:
      if word in string.punctuation:
        punctuationfree_list.append(" ")
      else :
        punctuationfree_list.append(word)
    return "".join(punctuationfree_list)


# def remove_punctuation(text):
#     punctuationfree = "".join([i for i in text if i not in string.punctuation])
#     return punctuationfree

In [80]:
# #defining function for tokenization
# 숫자 포함 토큰화
# def tokenization(text):
#     tokens = re.split('\W+', text)

#     return tokens

# 영어만 단어단위로 토큰화
p = re.compile('[a-z]+')

def tokenization(text):
    # print(text)
    result = p.findall(text)
    # print(result)
    return result

In [81]:
#stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')
# print(stopwords[:20]) 

#defining the function to remove stopwords from tokenized text
# stopwords 제거 
def remove_stopwords(text):
    for word in text:
      output= [word for word in text if word not in stopwords]
    return output

In [82]:
#setting lower case
comic['Lower_Summary'] = comic['Summary'].apply(lambda x: x.lower())
comic['Lower_Summary'].head(3)

0    she's young, single and about to achieve her d...
1    after binge-watching beauty videos online, a s...
2    after making a grisly discovery in the country...
Name: Lower_Summary, dtype: object

In [83]:
comic['No_Contraction_Summary'] = comic['Lower_Summary'].apply(lambda x:regularize_contractions(x))
comic['No_Contraction_Summary'].head(3)

0    she is young, single and about to achieve her ...
1    after binge-watching beauty videos online, a s...
2    after making a grisly discovery in the country...
Name: No_Contraction_Summary, dtype: object

In [84]:
comic['Clean_Summary'] = comic['No_Contraction_Summary'].apply(lambda x:remove_punctuation(x))
comic['Clean_Summary'].head(3)

0    she is young  single and about to achieve her ...
1    after binge watching beauty videos online  a s...
2    after making a grisly discovery in the country...
Name: Clean_Summary, dtype: object

In [85]:
#applying function to the column
comic['Tokenied_Summary'] = comic['Clean_Summary'].apply(lambda x: tokenization(x))

comic['Tokenied_Summary'].head(3)

0    [she, is, young, single, and, about, to, achie...
1    [after, binge, watching, beauty, videos, onlin...
2    [after, making, a, grisly, discovery, in, the,...
Name: Tokenied_Summary, dtype: object

In [86]:
#applying the function
comic['No_Stopwords_Summary'] = comic['Tokenied_Summary'].apply(lambda x:remove_stopwords(x))

comic['No_Stopwords_Summary'].head(3)

0    [young, single, achieve, dream, creating, incr...
1    [binge, watching, beauty, videos, online, shy,...
2    [making, grisly, discovery, countryside, small...
Name: No_Stopwords_Summary, dtype: object

In [87]:
def extract_no_verb(text):
  part_of_speech_text = pos_tag(text)
  result = []
  for word in part_of_speech_text:
    if word[1] != "VB" and word[1] != "VBD" and word[1] != "VBG" and word[1] != "VBN" and word[1] != "VBP" and word[1] != "VBZ":
        result.append(word[0])
  return result; 

comic['No_Verb_Summary'] = comic['No_Stopwords_Summary'].apply(lambda x:extract_no_verb(x))

comic['No_Verb_Summary'].head(3)

0    [young, single, achieve, dream, incredible, vi...
1    [binge, beauty, videos, online, shy, comic, bo...
2    [grisly, discovery, countryside, small, town, ...
Name: No_Verb_Summary, dtype: object

In [88]:
# df1 = comic['No_Stopwords_Summary']
# doc_words_list = df1.values.tolist()

# print(doc_words_list)


def wordlist_to_string(words_list) :
  doc = []
  for doc_word in words_list :
    doc.append(doc_word)
  return " ".join(doc)


#comic['Summary_final'] = comic['Stemmed_Summary'].apply(lambda x:wordlist_to_string(x))
comic['Summary_final'] = comic['No_Verb_Summary'].apply(lambda x:wordlist_to_string(x))

comic['Summary_final'].head(3)

0    young single achieve dream incredible videogam...
1    binge beauty videos online shy comic book fan ...
2    grisly discovery countryside small town book e...
Name: Summary_final, dtype: object

In [89]:
def extract_keywords(doc):
  keyword_list = []
  keyword_string = keywords(doc)
  keyword_list = keyword_string.split('\n')

  return keyword_list;


comic['Summary_Keywords'] = comic['Summary_final'].apply(lambda x:extract_keywords(x))

In [90]:
def keyword_3(keyword_list):
  keyword_list_3 = keyword_list[0:3]

  return keyword_list_3

comic['Summary_Keywords_3'] = comic['Summary_Keywords'].apply(lambda x:keyword_3(x))

comic.head(3)

,id,Name,Writer,Likes,Genre,Rating,Subscribers,Summary,Update,Reading Link,Lower_Summary,No_Contraction_Summary,Clean_Summary,Tokenied_Summary,No_Stopwords_Summary,No_Verb_Summary,Summary_final,Summary_Keywords,Summary_Keywords_3
0,0,Let's Play,Leeanne M. Krecic (Mongie),30.6M,Romance,9.62,4.2M,"She's young, single and about to achieve her d...",UP EVERY TUESDAY,https://www.webtoons.com/en/romance/letsplay/l...,"she's young, single and about to achieve her d...","she is young, single and about to achieve her ...",she is young single and about to achieve her ...,"[she, is, young, single, and, about, to, achie...","[young, single, achieve, dream, creating, incr...","[young, single, achieve, dream, incredible, vi...",young single achieve dream incredible videogam...,"[popular, videogames, single, stay, troublesome]","[popular, videogames, single]"
1,1,True Beauty,Yaongyi,39.9M,Romance,9.60,6.4M,"After binge-watching beauty videos online, a s...",UP EVERY WEDNESDAY,https://www.webtoons.com/en/romance/truebeauty...,"after binge-watching beauty videos online, a s...","after binge-watching beauty videos online, a s...",after binge watching beauty videos online a s...,"[after, binge, watching, beauty, videos, onlin...","[binge, watching, beauty, videos, online, shy,...","[binge, beauty, videos, online, shy, comic, bo...",binge beauty videos online shy comic book fan ...,"[secret, beauty videos, status short]","[secret, beauty videos, status short]"
2,2,Midnight Poppy Land,Lilydusk,10.4M,Romance,9.81,2.1M,After making a grisly discovery in the country...,UP EVERY SATURDAY,https://www.webtoons.com/en/romance/midnight-p...,after making a grisly discovery in the country...,after making a grisly discovery in the country...,after making a grisly discovery in the country...,"[after, making, a, grisly, discovery, in, the,...","[making, grisly, discovery, countryside, small...","[grisly, discovery, countryside, small, town, ...",grisly discovery countryside small town book e...,[deeper dangerous underworld],[deeper dangerous underworld]


In [91]:
from nltk.corpus import wordnet
nltk.download('wordnet')

#synsets(검색) -> 리스트

#synset(신셋색인)  .definition() .examples()  .lemmas()  .hyponyms()
# wordnet.synset('young.a.01').definition()
# wordnet.synset('beauty.n.01').hypernym_paths()
# word_beautiful = wordnet.synset('beautiful.a.01')
# word_pretty = wordnet.synset('pretty.a.01')
# print(word_pretty.wup_similarity(word_beautiful))

# dog = wordnet.synset('dog.n.01')
# cat = wordnet.synset('cat.n.01')

# print(dog.wup_similarity(cat))


!pip install textblob


# pos_tag(["beauty"])

# wordnet.synset('car.n.01').definition() wordnet.synset('car.n.01').examples() wordnet.synset('car.n.01').lemmas()

# comic['Summary_Keywords_3'] = comic['Summary_Keywords'].apply(lambda x:keyword_3(x))


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [101]:
from textblob import Word
import nltk
nltk.download('wordnet')
# word = Word("beauty")
# print(word.synsets)

def keyword_synsets(keywords):
  keyword_synsets = []
  for i in range(len(keywords)):
    keyword_synsets.append(keywords[i])
    word = Word(keywords[i])
    for synset in word.synsets:
      for lemma in synset.lemmas():
        if lemma.name() not in keyword_synsets:
          keyword_synsets.append(lemma.name())

  return keyword_synsets


comic['Summary_Keywords_Synsets'] = comic['Summary_Keywords'].apply(lambda x:keyword_synsets(x))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [102]:
comic['Summary_Keywords'].head(1)
comic['Summary_Keywords_Synsets'].head(1)

0    [popular, democratic, pop, videogames, single,...
Name: Summary_Keywords_Synsets, dtype: object

In [103]:
comic_df = comic[['id','Name', 'Writer', 'Genre', 'Rating', 'Summary_Keywords',
                 'Summary_Keywords_3', 'Summary_Keywords_Synsets']]
comic_df.head(1)

,id,Name,Writer,Genre,Rating,Summary_Keywords,Summary_Keywords_3,Summary_Keywords_Synsets
0,0,Let's Play,Leeanne M. Krecic (Mongie),Romance,9.62,"[popular, videogames, single, stay, troublesome]","[popular, videogames, single]","[popular, democratic, pop, videogames, single,..."


In [104]:
from sklearn.feature_extraction.text import CountVectorizer

comic_df['Summary_literal'] = comic_df['Summary_Keywords_Synsets'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
Summary_mat = count_vect.fit_transform(comic_df['Summary_literal'])
print(Summary_mat.shape)

(569, 14392)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [96]:
# 569개의 레코드, 2356개의 개별 단어 피처

In [105]:
from sklearn.metrics.pairwise import cosine_similarity

Summary_sim = cosine_similarity(Summary_mat, Summary_mat)
print(Summary_sim.shape)
print(Summary_sim[:2])

(569, 569)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [106]:
Summary_sim_sorted_ind = Summary_sim.argsort()[:, ::-1]
print(Summary_sim_sorted_ind[:1])

[[  0  55 129   4 301 169 248 195 402 115 201  47 269 331 272 213 389 338
  124 351 335 406 340 132  93 118  82 314 240 256  63 521 492 419  52 131
  182 320 394 415 232 497 318 253  28 424 153 179 185 183 181 184 180 156
  186 187 188 189 190 191 192 193 194 266 196 152 178 154 177 167 271 158
  159 160 161 162 163 164 165 270 166 168 176 155 198 268 170 171 172 173
  157 174 175 267 197 259 199 227 229 230 231 262 233 234 235 236 237 238
  239 241 242 243 244 245 246 247 261 260 249 250 251 252 254 255 257 228
  263 200 226 265 202 203 204 205 258 206 207 208 209 210 211 212 150 214
  215 216 217 218 219 220 221 264 222 223 224 225 151 142 149  44  51  50
   49  48  46  45  43 148  42  41  40  39  38  37  53  54  56  57  58  59
   60  61  62  64  65  66  67  68  69  70  71  36  35  34  16   1   2   3
    5   6   7   8   9  10  11  12  13  14  15  17  33  18  19  20  21  22
   23  24  25  26  27  29  30  31  32  72  73  74 130 112 113 114 116 117
  119 120 121 122 123 125 126 127 128 

In [107]:
def find_sim_comic(df, sorted_ind, c_name, top_n=10):
    comic_name = df[df['Name'] == c_name]
    comic_index = comic_name.index.values
    similar_indexes = sorted_ind[comic_index, :(top_n*5)]
    similar_indexes = similar_indexes.reshape(-1)
    print(similar_indexes)
    return df.iloc[similar_indexes][:top_n]
    # 마지막 대신 이거 쓰면 return df.iloc[similar_indexes].sort_values('Rating', ascending=False)[:top_n] 평점순으로

In [109]:
# 'Death of a Pop Star'와 비슷한 키워드 가진 웹툰 10개 추천
similar_comic = find_sim_comic(comic_df, Summary_sim_sorted_ind, 'Death of a Pop Star',10)
similar_comic[['Name', 'Rating' , 'Summary_Keywords', 'Summary_Keywords_Synsets', 'Summary_Keywords_3']]

[ 30 153 357 220 146 238 272 452 310 334 470 174   6  72 265 180 545  16
 241  80  22 379 520 519  96 465 345  29  74 375  42 101 540   1 248  93
 222 185  78 450  10 483 126  41 337 182   8 195 253 115]


,Name,Rating,Summary_Keywords,Summary_Keywords_Synsets,Summary_Keywords_3
30,Death of a Pop Star,9.34,"[reaper, lucky, build, dead]","[reaper, harvester, Grim_Reaper, Reaper, lucky...","[reaper, lucky, build]"
153,Metaphorical HER,8.95,"[builds, visual, criminals]","[builds, physique, build, body-build, habitus,...","[builds, visual, criminals]"
357,UndeadEd,9.69,"[dead, things]","[dead, all_in, beat, bushed, numb, deadened, i...","[dead, things]"
220,When Jasy Whistles,9.76,"[times, asuncion, dead, jasy]","[times, multiplication, time, clip, clock_time...","[times, asuncion, dead]"
146,Haxor,9.11,"[wrong, frame]","[wrong, wrongfulness, legal_injury, damage, in...","[wrong, frame]"
238,Lovely Hell,9.09,"[earth, powerful, figure]","[earth, Earth, world, globe, ground, land, dry...","[earth, powerful, figure]"
272,The Wolf & Red Riding Hood,9.63,"[anna, super, figure, school]","[anna, super, superintendent, ace, A-one, crac...","[anna, super, figure]"
452,Kind of Love,9.64,[short],"[short, short_circuit, shortstop, short-change...",[short]
310,Firebrand,8.92,"[natali, world politically, perfectly, evil, m...","[natali, world politically, perfectly, absolut...","[natali, world politically, perfectly]"
334,So I Married the Anti-Fan,9.42,"[hate house, suddenly, truth]","[hate house, suddenly, all_of_a_sudden, of_a_s...","[hate house, suddenly, truth]"
